In [20]:
import json
from zipfile import ZipFile
import pandas as pd

In [10]:
vid_info = list()

with open("data/video_info/esl_lol_vids.json", "r") as info_file:
    vid_info = json.load(info_file)
    vid2ind = {v["id"]:i for i, v in enumerate(vid_info)}

In [11]:
vid_info[vid2ind["83571434"]]

{'id': '83571434',
 'stream_id': '22827014832',
 'user_id': '30707866',
 'user_login': 'esl_lol',
 'user_name': 'ESL_LOL',
 'title': 'RERUN: fnatic vs. Millenium - Group A - IEM Katowice 2014 - League of Legends',
 'description': '',
 'created_at': '2016-08-14T14:26:23Z',
 'published_at': '2016-08-14T14:26:23Z',
 'url': 'https://www.twitch.tv/videos/83571434',
 'thumbnail_url': 'https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/ad3ef727c7_esl_lol_22827014832_500493871/thumb/thumb0-%{width}x%{height}.jpg',
 'viewable': 'public',
 'view_count': 36,
 'language': 'en',
 'type': 'archive',
 'duration': '18h30m39s',
 'muted_segments': None}

In [23]:
zf = ZipFile("./data/videos_chat/esl_lol_vids_chat.zip")
for f_name in zf.namelist():
    # print(f_name)if f_name.endswith("json") else None
    if f_name.endswith("json"):
        with zf.open(f_name, "r") as in_file:
            vid_chat = json.load(in_file)
            v_ind = vid2ind[f_name.split("/")[-1].strip(".json")]
            chat_info = {
                "msg_count": len(vid_chat["comments"]),
                "is_rerun": vid_info[v_ind]["title"].startswith("RERUN") or vid_info[v_ind]["title"].startswith("REBROADCAST")
            }
            
            vid_info[v_ind].update(chat_info)

In [24]:
vid_info_filtered = [vinf for vinf in vid_info if "msg_count" in vinf.keys()]

In [43]:
cols = ["id", "title", "created_at", "msg_count", "is_rerun", "duration"]

df_vid_info = pd.DataFrame(vid_info, columns=cols)
df_vid_info["created_at"] = pd.to_datetime(df_vid_info["created_at"])

df_vid_info

,id,title,created_at,msg_count,is_rerun,duration
0,1136665862,🔴 LIVE - ESL Mobile Open presents Wild Rift Oc...,2021-09-02 07:18:20+00:00,1282,False,8h11m38s
1,1129853487,🔴 LIVE - ESL Mobile Open presents Wild Rift Oc...,2021-08-26 07:14:01+00:00,376,False,5h46m43s
2,1122864123,🔴 LIVE - ESL Mobile Open presents Wild Rift Oc...,2021-08-19 07:15:57+00:00,1040,False,6h11m58s
3,1115898498,🔴 LIVE - ESL Mobile Open presents Wild Rift Oc...,2021-08-12 09:42:37+00:00,304,False,5h36m13s
4,1115852655,🔴 LIVE - ESL Mobile Open presents Wild Rift Oc...,2021-08-12 07:31:14+00:00,256,False,2h10m39s
...,...,...,...,...,...,...
1015,38088822,CPL Shenyang Korean Qualifier. Tempest vs CJ E...,2012-08-13 11:48:06+00:00,0,False,3h39m30s
1016,38088808,Go4LoL EU West June Final with Joe and ButButB...,2012-08-09 17:56:24+00:00,0,False,6h18m57s
1017,38088787,ESL Pro Series LoL Playoffs with Joe and ButBu...,2012-08-04 15:02:40+00:00,0,False,4h49m30s
1018,38088203,Gamescom European Regional Finals Decider,2012-08-02 18:57:41+00:00,0,False,1h59m59s


In [44]:
df_vid_info.describe()

,msg_count
count,1020.000000
mean,2107.175490
std,16294.075474
min,0.000000
25%,0.000000
50%,1.000000
75%,265.000000
max,264144.000000


In [39]:
df_commented_vids = df_vid_info[df_vid_info.msg_count > 0]
f"{df_commented_vids.id.count()}/{df_vid_info.id.count()} videos with comments"

'511/1020 videos with comments'

In [47]:
df_reruns = df_vid_info[df_vid_info.is_rerun]
f"{df_reruns.id.count()}/{df_vid_info.id.count()} videos are reruns"

'652/1020 videos are reruns'

In [40]:
df_commented_vids.msg_count.mean()

4206.103718199609

In [45]:
df_vid_info["created_at"].min(), df_vid_info["created_at"].max()

(Timestamp('2012-05-29 11:43:41+0000', tz='UTC'),
 Timestamp('2021-09-02 07:18:20+0000', tz='UTC'))